In [2]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.layouts import column, gridplot, row
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Select
from bokeh.models.callbacks import CustomJS
import numpy as np
import scipy.stats

output_notebook()

Loading BokehJS ...

In [3]:
column_labels = ["Channel","Region","Fresh","Milk","Grocery","Frozen","Detergents_Paper","Delicassen"]
complete_data = np.loadtxt("Wholesale_customers data.csv", dtype=int, skiprows=1, delimiter=',')

In [176]:
with open("Wholesale_customers_data-missing.csv") as f:
    next(f)
    r=1
    rc_list=[] #row,column tuple of missing value
    missing = 0
    m_data= []
    for line in f:
        l = line.strip('\n').split(',')
        for i,val in enumerate(l):
            if not val:
                missing = missing +1
                tup = (r,i)
                rc_list.append(tup)
                l[i]=0
        m_data.append(tuple(l))
        r=r+1
    print("%d missing values found on (row,col): "%(missing)+" ".join(str(t) for t in rc_list))
missing_data = np.asarray(m_data, dtype=float)
#remove rows with missing data, try linear regression on each column that is missing
clean_data = np.delete(missing_data,[76,173,181,226,275,371],0)

actual_values = dict()
for tup in rc_list:
    actual_values[tup] = complete_data[tup[0],tup[1]]
    print("At %s actual value is %d"%(str(tup),actual_values[tup]))

6 missing values found on (row,col): (76, 4) (173, 4) (181, 3) (226, 6) (275, 7) (371, 6)
At (76, 4) actual value is 6532
At (173, 4) actual value is 19805
At (181, 3) actual value is 29627
At (226, 6) actual value is 2568
At (275, 7) actual value is 379
At (371, 6) actual value is 716


In [178]:
CHANNEL = 0
REGION = 1
FRESH = 2
MILK = 3
GROCERY = 4
FROZEN = 5
DETERGENTS =6
DELICASSEN = 7

def plot_distribution(attr_name, calculated_attr_column,clean_attr_column):
    
    clean_density, clean_edge = np.histogram(clean_attr_column, density=True,bins=40)
    clean_unity_density = clean_density/clean_density.sum()
 
    
    calculated_density, calculated_edge = np.histogram(calculated_attr_column, density=True,bins=40)
    calculated_unity_density = calculated_density/calculated_density.sum()

    
    mu, sigma = np.mean(clean_attr_column), np.std(clean_attr_column)
    p1 = figure(title="[Clean]Dist. of ann. $ on %s"%(attr_name),background_fill_color="white")
    p1.quad(top=clean_unity_density, bottom=0, left=clean_edge[:-1], right=clean_edge[1:], fill_color='blue', line_color='blue')
    p1.xaxis.axis_label = '%s (μ=%0.2f, σ=%0.2f)'%(attr_name,mu,sigma)
    p1.yaxis.axis_label = 'Prob(spend on %s)'%(attr_name)
    
    mu, sigma = np.mean(calculated_attr_column), np.std(calculated_attr_column)
    p2 = figure(title="[Calculated]Dist. of ann. $ on %s"%(attr_name),background_fill_color="white")
    p2.quad(top=calculated_unity_density, bottom=0, left=calculated_edge[:-1], right=calculated_edge[1:], fill_color='blue', line_color='blue')
    p2.xaxis.axis_label = 'Annual spending on %s (μ=%0.2f, σ=%0.2f)'%(attr_name,mu,sigma)
    p2.yaxis.axis_label = 'Prob(spend on %s)'%(attr_name)
    return p1,p2

In [183]:
#Method 1: Take a simple average of each column, use this in place of missing value
missing_cols = set(t[1] for t in rc_list)
col_avg = dict()
missing_col_counts = dict()

for t in rc_list:
    missing_col_counts[t[1]] = missing_col_counts.get(t[1], 0) + 1

for col in missing_cols:
    c = missing_data[:,col]
    avg = sum(c)/(len(c) - missing_col_counts[col])
    col_avg[col] = avg
    print("col %d avg is %.2f"%(col,avg))
    #replace values manually
    if col == 3:
        missing_data[181,3] = avg
    if col == 4:
        missing_data[76,4] = avg
        missing_data[173,4] = avg
    if col == 6:
        missing_data[226,6] = avg
        missing_data[371,6] = avg
    if col == 7:
        missing_data[275,7] = avg
rmse_meanvals = rmse(actual_vals, mean_vals)

affected_cols = {
    "Milk": (missing_data[:,MILK],clean_data[:,MILK]),
    "Grocery": (missing_data[:,GROCERY],clean_data[:,MILK]),
    "Detergents/Paper": (missing_data[:,DETERGENTS],clean_data[:,MILK]),
    "Deelicassen":(missing_data[:,DELICASSEN],clean_data[:,MILK])
}
p = [None]*8
for i, attr_name in enumerate(affected_cols.keys()):
    calculated_attr_column,clean_attr_column = affected_cols[attr_name][0],affected_cols[attr_name][1]
    p[2*i], p[2*i+1] = plot_distribution(attr_name, calculated_attr_column,clean_attr_column)
    
show(gridplot(p,  ncols=2, plot_width=400, plot_height=400))

#Left: distribution of data with rows containing missing values removed
#Right: distribution of data with missing values replaced by column average

col 3 avg is 5766.50
col 4 avg is 7963.53
col 6 avg is 2892.58
col 7 avg is 1525.56


In [184]:
#Method 2: Use linear regression, model other parameters -> predict column of missing value
from sklearn.linear_model import LinearRegression
for col in missing_cols:
    print("predicting in column %d"%(col))
    #extract the column we are trying to predict
    y_train = clean_data[:,col]
    #remove the column we are trying to predict from the dat
    X_train = np.delete(clean_data, col ,1)
    
    #get the rows that have missing vals for this column
    rows_to_predict = [t[0] for t in rc_list if t[1] == col]
    missing_rows = missing_data[rows_to_predict]
    X_test = np.delete(missing_rows, col, 1)
    
    #train model
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    #predict missing values 
    y_pred_lr = lr.predict(X_test)
    
    av = [actual_values[(r,col)] for r in rows_to_predict]
    
    print("predicted: "+str(y_pred_lr) + " actual values "+ str(av))
    if col == 3:
        missing_data[181,3] = y_pred_lr[0]
    if col == 4:
        missing_data[76,4] = y_pred_lr[0]
        missing_data[173,4] = y_pred_lr[1]
    if col == 6:
        missing_data[226,6] = y_pred_lr[0]
        missing_data[371,6] = y_pred_lr[1]
    if col == 7:
        missing_data[275,7] = y_pred_lr[0]
        
affected_cols = {
    "Milk": (missing_data[:,MILK],clean_data[:,MILK]),
    "Grocery": (missing_data[:,GROCERY],clean_data[:,MILK]),
    "Detergents/Paper": (missing_data[:,DETERGENTS],clean_data[:,MILK]),
    "Deelicassen":(missing_data[:,DELICASSEN],clean_data[:,MILK])
}
p = [None]*8
for i, attr_name in enumerate(affected_cols.keys()):
    calculated_attr_column,clean_attr_column = affected_cols[attr_name][0],affected_cols[attr_name][1]
    p[2*i], p[2*i+1] = plot_distribution(attr_name, calculated_attr_column,clean_attr_column)
    
show(gridplot(p,  ncols=2, plot_width=400, plot_height=400))


#Left: distribution of data with rows containing missing values removed
#Right: distribution of data with missing values replaced by column average

predicting in column 3
predicted: [ 16800.20264878] actual values [29627]
predicting in column 4
predicted: [  3022.60187191  19315.51620873] actual values [6532, 19805]
predicting in column 6
predicted: [ 3941.98412068  -281.89007455] actual values [2568, 716]
predicting in column 7
predicted: [ 308.92353506] actual values [379]


In [181]:
#calculate rmse
import math
def rmse(y_true, y_pred):
    sq_err = 0
    for yt, yp in zip(y_true,y_pred):
        sq_err = sq_err + math.pow((yt-yp),2)
    mean_sq_err = sq_err/len(y_pred)
    rmse_ = math.sqrt(mean_sq_err)
    return rmse_

In [182]:
#replacing with mean
#written out by hand because i was running out of time haha
#values are organized by increasing columns (3,4,6,7) and then increasing row 
actual_vals = [29627, 6532, 19805, 2568, 716, 379]
mean_vals = [5795.72, 7972.66, 7972.66, 2891.72, 2891.72, 1525.72]
lr_vals = [16800.20264878, 3022.60187191, 19315.51620873, 3941.98412068, -281.89007455, 308.92353506] 

rmse_meanvals = rmse(actual_vals, mean_vals)
rmse_lr = rmse(actual_vals, lr_vals)

print("RMSE of replacing with mean value for column %0.2f"%(rmse_meanvals))
print("RMSE of replacing with predicted linear regression value for missing value in row %0.2f"%(rmse_lr))

#RMSE of using linear regression is smaller than when just replacing with column mean
#The distribution plots look closer to 'clean' data for linear regression than for replace with mean

RMSE of replacing with mean value for column 10925.22
RMSE of replacing with predicted linear regression value for missing value in row 5476.78
